In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss

In [2]:
#function for reading database, return a dataframe, return a dataframe
def readDB(csv_database):
    sql = 'select * from sensorProcessed' #sql need to be executed. 
    df = pd.read_sql_query(sql, csv_database) # read data as dataframe
    return df

In [3]:
#set credentials file for plotly(username & API key)
import plotly
username = 'mengyizhou95' #username registered on Ploty
api_key = 'X5DOiJATuUqPbsabrbxA' #the api key plotly asigned
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [4]:
#set the conversion ratio from min to ms
timeConversionVal = 60000

In [5]:
#function for trans time from min to ms
def MinToMS(time, df):
    time = time * timeConversionVal
    time = df.time.iloc[(df['time']-time).abs().argsort()[:1]] #find the closest time on table
    return time.values[0]

In [6]:
#function for generate flapper Chart
def flapperChart(df, start=None, end=None, filename='flapperChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=df.flapper0[startIndex: endIndex],
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper1[startIndex: endIndex],
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper2[startIndex: endIndex],
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper3[startIndex: endIndex],
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 255, 0)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper4[startIndex: endIndex],
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper5[startIndex: endIndex],
        name = 'flapper5',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper6[startIndex: endIndex],
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper7[startIndex: endIndex],
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'friction',
                                        range = [0, 100]))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [7]:
#function for generate diameter chart
def diameterChart(df, start=None, end=None, filename='diameterChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0] # get index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.diameter0[startIndex: endIndex],
        name = 'diameter0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.diameter1[startIndex: endIndex],
        name = 'diameter1',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )

    data = [trace0, trace1]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'diameter sensor output',
                                        range = [0, 100]))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [8]:
#function for generating orientation chart
def orientationChartOriginal(df, start=None, end=None, filename='orientationChart'):
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time.iloc[startIndex: endIndex]/timeConversionVal,
        y=df.orientationX.iloc[startIndex: endIndex],
        name = 'orientationX',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time.iloc[startIndex: endIndex]/timeConversionVal,
        y=df.orientationY.iloc[startIndex: endIndex],
        name = 'orientationY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time.iloc[startIndex: endIndex]/timeConversionVal,
        y=df.orientationZ.iloc[startIndex: endIndex],
        name = 'orientationZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2]
    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'orientation(degrees)'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [9]:
def accChart(df, start=None, end=None, filename='accChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accX[startIndex: endIndex],
        name = 'accX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accY[startIndex: endIndex],
        name = 'accY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accZ[startIndex: endIndex],
        name = 'accZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2]
    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'acceleration',))

    fig = dict(data = data, layout = layout)
    py.iplot(fig, filename = 'diameter sensor')
    plotly.offline.plot(fig, filename=filename + '.html')

In [10]:
def compassChart(df, start=None, end=None, filename='compassChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassX[startIndex: endIndex],
        name = 'compassX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassY[startIndex: endIndex],
        name = 'compassY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassZ[startIndex: endIndex],
        name = 'compassZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'compass sensor output',))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [6]:
#execute read database
# csv_database = 'sqlite:///SouthHarrison.db' #database path
csv_database = 'sqlite:///Cambridge1.db' 
df1 = readDB(csv_database)
csv_database = 'sqlite:///Cambridge2.db' 
df2 = readDB(csv_database)

In [47]:
csv_database = 'sqlite:///sa_run10.db' 
df1 = readDB(csv_database)

In [22]:
df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run13_trim.csv')
diff = df.time[17165] - df.time[14798]

In [23]:
df.time[17165: ] = df.time[17165: ] - diff
df.drop(df.index[14798: 17165], inplace=True)

In [24]:
df = df.reset_index(drop=True)

In [25]:
df.to_csv("sa_run13_trim_new.csv")

In [ ]:
#concat two parts of df1

diff = start2 - end1
dftmp1 = dftmp1.reset_index(drop=True)
dftmp2 = dftmp2.reset_index(drop=True)
dftmp2["time"] = dftmp2["time"].apply(lambda x: x - diff)

df1 = pd.concat([dftmp1, dftmp2])
df1 = df1.reset_index(drop=True)

In [48]:
#select two range of data from run1
start1 = 32.77327
end1 = 38.76188
start2 = 67.85277
end2 = 75.3671
start3 = 77.03842
end3 = 80.29525

start1 = MinToMS(start1, df1)
end1 = MinToMS(end1, df1)
start2 = MinToMS(start2, df1)
end2 = MinToMS(end2, df1)

start3 = MinToMS(start3, df1)
end3 = MinToMS(end3, df1)

startIndex1 = df1.index[df1['time'] == start1].tolist()[0]
endIndex1 = df1.index[df1['time'] == end1].tolist()[0]
startIndex2 = df1.index[df1['time'] == start2].tolist()[0]
endIndex2 = df1.index[df1['time'] == end2].tolist()[0]

startIndex3 = df1.index[df1['time'] == start3].tolist()[0]
endIndex3 = df1.index[df1['time'] == end3].tolist()[0]

dftmp1 = df1.iloc[startIndex1:endIndex1]
dftmp2 = df1.iloc[startIndex2:endIndex2]

dftmp3 = df1.iloc[startIndex3:endIndex3]

In [49]:
#concat two parts of df1
diff = start2 - end1
dftmp1 = dftmp1.reset_index(drop=True)
dftmp2 = dftmp2.reset_index(drop=True)
dftmp2["time"] = dftmp2["time"].apply(lambda x: x - diff)

# diff2 = start3 - end2
# dftmp3 = dftmp3.reset_index(drop=True)
# dftmp3["time"] = dftmp3["time"].apply(lambda x: x - diff - diff2)

df1 = pd.concat([dftmp1, dftmp2])
# df1 = pd.concat([df1, dftmp3])
df1 = df1.reset_index(drop=True)

In [51]:
first = df1['time'].head(1).tolist()[0]
df1['time'] = df1['time'] - first

df1['time'].head(5)

0      0
1     32
2     69
3    101
4    133
Name: time, dtype: int64

In [52]:
df1.to_csv("sa_run10_trim.csv")

In [16]:
#trim data for run1

# flapperChart(df1, filename='flapperTrim01')
# diameterChart(df1, filename='diameterTrim01')
# orientationChartOriginal(df1, filename='orientationTrim01')
# accChart(df1, filename='accelerationTrim01')
compassChart(df1, filename='compassTrim01')

In [146]:
#trim data for run2
start = 9.691133
end = 16.70995

flapperChart(df2, start, end, 'flapperTrim02')
diameterChart(df2, start, end, 'diameterTrim02')
orientationChartOriginal(df2, start, end, 'orientationTrim02')
accChart(df2, start, end, 'accelerationTrim02')
compassChart(df2, start, end, 'compassTrim02')

In [11]:
order = 5 #order of filter (1-)
fs = 50       # sample rate, Hz
cutoff = 10 # desired cutoff frequency of the filter, Hz

In [7]:
#functino for low/high pass filter
from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [13]:
#function for generate flapper Chart
def flapperChartLowpass(df, start=None, end=None, filename='flapperChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=butter_lowpass_filter(df.flapper0[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper1[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper1',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper2[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper2',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper3[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 255, 0)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper4[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper5[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper5',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper6[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper7[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'flapper sensor output (0 - 100)'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [9]:
#function for generate flapper Chart
def flapperChartHighpass(df, start=None, end=None, filename='flapperChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=butter_highpass_filter(df.flapper0[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper1[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper2[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper3[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 255, 0)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper4[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper5[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper5',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper6[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_highpass_filter(df.flapper7[startIndex: endIndex], cutoff, fs, order),
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'flapper sensor output (0 - 100)'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [15]:
start = 9.687217
end = 16.05

flapperChartLowpass(df2, start, end, 'flapperTrimLowpass02')
# flapperChartHighpass(df2, start, end, 'flapperTrimHighpass02')

In [12]:
csv_database = 'sqlite:///SouthHarrison.db' 
df = readDB(csv_database)
start = 0
end = 180
flapperChartLowpass(df, 0, 180, 'flapperSHLowpass')

In [10]:
#slice df2
start = 9.691133
end = 16.70995
start = MinToMS(start, df2)
end = MinToMS(end, df2)
startIndex = df2.index[df2['time'] == start].tolist()[0] #get the index of start & end
endIndex = df2.index[df2['time'] == end].tolist()[0]

df2 = df2.iloc[startIndex:endIndex]

In [11]:
#time
tp1 = df1.iloc[0]['time'].item()
tp2 = df2.iloc[0]['time'].item()
df1["time"] = df1["time"].apply(lambda x: x - tp1)
df2["time"] = df2["time"].apply(lambda x: x - tp2)

In [12]:
df1 = df1.drop(columns=['index'])
df2 = df2.drop(columns=['index'])

df2 = df2.reset_index(drop=True)

In [13]:
#rename flapper
df1.columns = ['time', \
                    'front-90', 'front-0', 'front-270', 'front-180', 'back-315', 'back-45', 'back-135', 'back-225', \
                    'diameter0', 'diameter1', \
                    'orientationX', 'orientationY', 'orientationZ', \
                    'accX', 'accY', 'accZ', \
                    'compassX', 'compassY', 'compassZ']
df2.columns = ['time', \
                    'front-90', 'front-0', 'front-270', 'front-180', 'back-315', 'back-45', 'back-135', 'back-225', \
                    'diameter0', 'diameter1', \
                    'orientationX', 'orientationY', 'orientationZ', \
                    'accX', 'accY', 'accZ', \
                    'compassX', 'compassY', 'compassZ']

In [14]:
def showChart(df, filename='chart'):
    
    trace0 = go.Scattergl(
        x=df.time/timeConversionVal, 
        y=df['front-90'],
        name = 'front-90',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['front-0'],
        name = 'front-0',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['front-270'],
        name = 'front-270',
        line = dict(
            color = ('rgb(60, 60, 225)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['front-180'],
        name = 'front-180',
        line = dict(
            color = ('rgb(60, 225, 60)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['back-315'],
        name = 'back-315',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['back-45'],
        name = 'back-45',
        line = dict(
            color = ('rgb(60, 225, 255)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['back-135'],
        name = 'back-135',
        line = dict(
            color = ('rgb(225, 125, 0)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df['back-225'],
        name = 'back-225',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
#     ###################################
    trace8 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.orientationX,
        name = 'orientationX',
        line = dict(
            color = ('rgb(225, 60, 60)'),
            width = 1)
    )
    trace9 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.orientationY,
        name = 'orientationY',
        line = dict(
            color = ('rgb(60, 60, 225)'),
            width = 1)
    )
    trace10 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.orientationZ,
        name = 'orientationZ',
        line = dict(
            color = ('rgb(60, 225, 60)'),
            width = 1)
    )
    ########################################
    trace11 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.accX,
        name = 'accX',
        line = dict(
            color = ('rgb(225, 60, 60)'),
            width = 1)
    )
    trace12 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.accY,
        name = 'accY',
        line = dict(
            color = ('rgb(60, 60, 225)'),
            width = 1)
    )
    trace13 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=df.accZ,
        name = 'accZ',
        line = dict(
            color = ('rgb(60, 225, 60)'),
            width = 1)
    )
    
    fig = tools.make_subplots(rows=3, cols=1, subplot_titles=('Flapper', 'Orientation', 'Acceleration'))
    ##########################
    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 1)
    fig.append_trace(trace3, 1, 1)
    fig.append_trace(trace4, 1, 1)
    fig.append_trace(trace5, 1, 1)
    fig.append_trace(trace6, 1, 1)
    fig.append_trace(trace7, 1, 1)
    
    fig.append_trace(trace8, 2, 1)
    fig.append_trace(trace9, 2, 1)
    fig.append_trace(trace10, 2, 1)
    
    fig.append_trace(trace11, 3, 1)
    fig.append_trace(trace12, 3, 1)
    fig.append_trace(trace13, 3, 1)
    
#     data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    fig['layout']['xaxis1'].update(title='time(minutes)', gridcolor=('rgb(125, 125, 125)'))
    fig['layout']['xaxis2'].update(title='time(minutes)', gridcolor=('rgb(125, 125, 125)'))
    fig['layout']['xaxis3'].update(title='time(minutes)', gridcolor=('rgb(125, 125, 125)'))

    fig['layout']['yaxis1'].update(title='friction', range=[0, 100], gridcolor=('rgb(125, 125, 125)'))
    fig['layout']['yaxis2'].update(title='orientation(degrees)', gridcolor=('rgb(125, 125, 125)'))
    fig['layout']['yaxis3'].update(title='acceleration(0.1 m/s)', gridcolor=('rgb(125, 125, 125)'))
    
    fig['layout'].update(title='Test in Cambridge')

#     fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [15]:
from plotly import tools

In [16]:
showChart(df1, 'AllinOne01')
showChart(df2, 'AllinOne02')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



In [18]:
df1.to_csv('trimedData01.csv')
df2.to_csv('trimedData02.csv')